<a href="https://colab.research.google.com/github/sravanreddy6745/Traffic-Sign-Classification/blob/master/optiizecnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import  matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle 
import random
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
os.getcwd()

'/content'

In [4]:
with open('/content/drive/My Drive/Traffic-Sign-Classification/traffic-signs-data/train.p', 'rb') as f:
    train_data = pickle.load(f)
    
with open('/content/drive/My Drive/Traffic-Sign-Classification/traffic-signs-data/valid.p', 'rb') as f:
    val_data = pickle.load(f)
with open('/content/drive/My Drive/Traffic-Sign-Classification/traffic-signs-data/test.p', 'rb') as f:
    test_data = pickle.load(f)

In [5]:
X_train, y_train = train_data['features'], train_data['labels']
X_val, y_val = val_data['features'], val_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

OptimizeCNNModel

In [6]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=146d5bc594df76293cf9d2fa8bde2d3ba94deabbca67ce8ccac988a24a2eb587
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=cc7b8abaac809b3ffa1c8153581b837ff5771e0a5d69273db6b6298dc65f1c9d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
X_train=X_train/255.0
X_test=X_test/255.0

In [9]:
X_train[0].shape

(32, 32, 3)

In [10]:
X_train=X_train.reshape(len(X_train),32,32,3)
X_test=X_test.reshape(len(X_test),32,32,3)

In [11]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        padding='same',
        input_shape=(32,32,3)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu',
            padding='same'
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(
            filters=hp.Int('conv_3_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
            activation='relu',
            padding='same'
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu',
            padding='same'
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(rate=hp.Choice('drop',values=[0.2,0.3])),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=124, max_value=1024, step=16),
            activation='relu'
        ),
        keras.layers.Dense(43, activation='softmax')
        ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model




In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='output',project_name="TrafficSign")

In [14]:
tuner_search.search(X_train,y_train,epochs=5,validation_data=(X_val,y_val))

Epoch 1/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.5252 - accuracy: 0.0547 - val_loss: 3.5751 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 8s 8ms/step - loss: 3.4844 - accuracy: 0.0553 - val_loss: 3.5682 - val_accuracy: 0.0476
Epoch 3/5
1088/1088 [==============================] - 8s 8ms/step - loss: 3.4825 - accuracy: 0.0538 - val_loss: 3.5637 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.4815 - accuracy: 0.0562 - val_loss: 3.5672 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.4807 - accuracy: 0.0540 - val_loss: 3.5735 - val_accuracy: 0.0476


Epoch 1/5
1088/1088 [==============================] - 12s 11ms/step - loss: 3.6472 - accuracy: 0.0556 - val_loss: 3.5645 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 11s 10ms/step - loss: 3.4840 - accuracy: 0.0551 - val_loss: 3.5518 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 11s 10ms/step - loss: 3.4824 - accuracy: 0.0565 - val_loss: 3.5525 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 11s 10ms/step - loss: 3.4813 - accuracy: 0.0570 - val_loss: 3.5537 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 11s 10ms/step - loss: 3.4804 - accuracy: 0.0570 - val_loss: 3.5562 - val_accuracy: 0.0476


Epoch 1/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.5866 - accuracy: 0.0544 - val_loss: 3.5658 - val_accuracy: 0.0478
Epoch 2/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.4846 - accuracy: 0.0553 - val_loss: 3.5726 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.4826 - accuracy: 0.0562 - val_loss: 3.5512 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.4812 - accuracy: 0.0547 - val_loss: 3.5650 - val_accuracy: 0.0469
Epoch 5/5
1088/1088 [==============================] - 9s 8ms/step - loss: 3.4803 - accuracy: 0.0560 - val_loss: 3.5610 - val_accuracy: 0.0544


Epoch 1/5
1088/1088 [==============================] - 8s 7ms/step - loss: 3.6567 - accuracy: 0.0546 - val_loss: 3.5554 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 7s 7ms/step - loss: 3.4844 - accuracy: 0.0536 - val_loss: 3.5678 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 7s 7ms/step - loss: 3.4824 - accuracy: 0.0569 - val_loss: 3.5580 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 7s 7ms/step - loss: 3.4813 - accuracy: 0.0569 - val_loss: 3.5563 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 7s 7ms/step - loss: 3.4805 - accuracy: 0.0548 - val_loss: 3.5582 - val_accuracy: 0.0544


Epoch 1/5
1088/1088 [==============================] - 10s 9ms/step - loss: 0.7533 - accuracy: 0.7855 - val_loss: 57.5617 - val_accuracy: 0.8778
Epoch 2/5
1088/1088 [==============================] - 9s 9ms/step - loss: 0.0740 - accuracy: 0.9787 - val_loss: 34.8230 - val_accuracy: 0.9154
Epoch 3/5
1088/1088 [==============================] - 9s 9ms/step - loss: 0.0409 - accuracy: 0.9877 - val_loss: 40.3195 - val_accuracy: 0.9011
Epoch 4/5
1088/1088 [==============================] - 9s 9ms/step - loss: 0.0370 - accuracy: 0.9894 - val_loss: 52.3354 - val_accuracy: 0.9014
Epoch 5/5
1088/1088 [==============================] - 10s 9ms/step - loss: 0.0309 - accuracy: 0.9909 - val_loss: 32.5966 - val_accuracy: 0.9324


...

Epoch 1/5
1088/1088 [==============================] - 11s 10ms/step - loss: 0.5906 - accuracy: 0.8296 - val_loss: 27.4227 - val_accuracy: 0.9236
Epoch 2/5
1088/1088 [==============================] - 10s 10ms/step - loss: 0.0601 - accuracy: 0.9827 - val_loss: 21.1873 - val_accuracy: 0.9544
Epoch 3/5
1088/1088 [==============================] - 10s 9ms/step - loss: 0.0407 - accuracy: 0.9879 - val_loss: 20.4609 - val_accuracy: 0.9472
Epoch 4/5
1088/1088 [==============================] - 10s 9ms/step - loss: 0.0417 - accuracy: 0.9882 - val_loss: 30.2109 - val_accuracy: 0.9506
Epoch 5/5
1088/1088 [==============================] - 10s 9ms/step - loss: 0.0242 - accuracy: 0.9926 - val_loss: 14.2705 - val_accuracy: 0.9635


Epoch 1/5
1088/1088 [==============================] - 8s 8ms/step - loss: 3.5889 - accuracy: 0.0549 - val_loss: 3.5548 - val_accuracy: 0.0549
Epoch 2/5
1088/1088 [==============================] - 8s 7ms/step - loss: 3.4833 - accuracy: 0.0568 - val_loss: 3.5555 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 8s 7ms/step - loss: 3.4827 - accuracy: 0.0546 - val_loss: 3.5613 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 8s 7ms/step - loss: 3.4817 - accuracy: 0.0563 - val_loss: 3.5529 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 8s 7ms/step - loss: 3.4807 - accuracy: 0.0560 - val_loss: 3.5629 - val_accuracy: 0.0546


Epoch 1/5
1088/1088 [==============================] - 12s 11ms/step - loss: 0.7032 - accuracy: 0.7952 - val_loss: 66.9269 - val_accuracy: 0.8390
Epoch 2/5
1088/1088 [==============================] - 12s 11ms/step - loss: 0.0713 - accuracy: 0.9799 - val_loss: 59.3692 - val_accuracy: 0.9057
Epoch 3/5
1088/1088 [==============================] - 12s 11ms/step - loss: 0.0434 - accuracy: 0.9864 - val_loss: 50.1447 - val_accuracy: 0.9179
Epoch 4/5
1088/1088 [==============================] - 11s 11ms/step - loss: 0.0357 - accuracy: 0.9900 - val_loss: 69.5992 - val_accuracy: 0.9107
Epoch 5/5
1088/1088 [==============================] - 12s 11ms/step - loss: 0.0324 - accuracy: 0.9906 - val_loss: 46.0752 - val_accuracy: 0.9240


Epoch 1/5
1088/1088 [==============================] - 10s 9ms/step - loss: 4.0022 - accuracy: 0.0540 - val_loss: 3.5623 - val_accuracy: 0.0476
Epoch 2/5
1088/1088 [==============================] - 10s 9ms/step - loss: 3.4845 - accuracy: 0.0550 - val_loss: 3.5549 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 10s 9ms/step - loss: 3.4828 - accuracy: 0.0543 - val_loss: 3.5528 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 10s 9ms/step - loss: 3.4818 - accuracy: 0.0567 - val_loss: 3.5563 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 10s 9ms/step - loss: 3.4810 - accuracy: 0.0553 - val_loss: 3.5540 - val_accuracy: 0.0544


INFO:tensorflow:Oracle triggered exit


In [25]:
model=tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 48)        3648      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       55424     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 80)        92240     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       92288     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 128)         0

In [27]:
score=model.evaluate(X_test,y_test)

395/395 [==============================] - 2s 4ms/step - loss: 0.1850 - accuracy: 0.9612


In [22]:
score


[0.1850128173828125, 0.9612034559249878]

In [44]:
model.save('/content/drive/My Drive/cnnoptiize')

INFO:tensorflow:Assets written to: /content/drive/My Drive/cnnoptiize/assets


In [29]:
best_hyperparameters = tuner_search.get_best_hyperparameters(1)[0]

In [42]:
for i in list(tuner_search.get_best_hyperparameters(1)[0]):
  print(i)

TypeError: ignored